# Earthquake Data Analysis

### Description

The catalog includes the magnitude, time of occurrence (s), and 3D coordinates (m) of earthquakes in about 20 years of recording in South California. Coordinates were converted from latitude, longitude, and depth of events in a seismic catalog. Magnitudes should be within the range $[0,8]$.

* **Waiting time (t)**: time interval between an event and the next one in the sequence.
* **Distance (r)**: Eucledian 3D distance between events. (each 3D set of coordinates refers to the hypocenter, i.e. the point triggering the slip in a fault that forms the earthquake)


### Assignments

1. Deduce what is the variable in each column of the catalog.
2. Visualize the process in space and/or time with suitable time series and/or 3D visualizations of the hypocenters. For instance, plot a space variable (a single coordinate or a nice linear combination of coordinates) as a function of time.
3. Compute the distribution $P_m(t)$ of waiting times for events of magnitude m or above (i.e. do not consider events below $m$). In shaping the bin sizes, take into account that this distribution is expected to have a power-law decay with time (e.g $\sim 1/t$), and that a power-law is well visualized in log-log scale. Do this analysis for many values of $m$, say $m=2,3,4,5$.
4. Compute the distribution $P_m(r)$ of the distance between an event and the next one, considering earthquakes of magnitude m or above. Also here make a clever choice for the bin sizes and try several values of $m$.
5. Compute the distribution $P_{m,R}(t)$ of waiting times for events of magnitude $m$ or above, which are separated by at most a distance $r<R$, for different values of m and $R$. (In this statistics, if the following event is farther than $R$, skip the $t$ and go to the next pair)
6. Eventually note if, from the analysis of the previous points, there emerges a scaling picture. Is there a suitable rescaling that collapses distributions for various $m$ (and eventually $R$ if point 5 is considered) on a single curve?

### Datasets

* column 1: index of the event
* column 2: index of the previous event that triggered it (defined with a given algorithm), -1 if no ancestor is found
* column 3: time (seconds) from 0:00 of Jan.1st, 1982
* column 4: magnitude
* columns 5, 6, and 7: 3D coordinates (meters) of the earthquake hypocenter, i.e. of the point from where it started. These Euclidean coordinates are derived from latitude, longitude and depth.

Joining each event to that with the index of the second column (if not -1), there emerges a set of causal trees.


### Contact
* Marco Baiesi <marco.baiesi@unipd.it>

In [ ]:
import pandas as pd

data = 'SouthCalifornia-1982-2011_Physics-of-Data.dat'


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#filepath = '/content/drive/MyDrive/LCP_project/'

In [ ]:
#df = pd.read_csv(filepath + data, sep = ' ', names = ['Index', 'Prev event', 'Time', 'Magnitude', 'x', 'y', 'z'])
df = pd.read_csv(data, sep = ' ', names = ['Index', 'Prev_event', 'Time', 'Magnitude', 'x', 'y', 'z'])

df.head(5)

In [ ]:
df.tail(5)
df

# Graphical Part

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(18,10))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=df['Time'], cmap='viridis', marker='.')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
fig.colorbar(scatter1, label='Time', shrink=0.7)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=df['z'], cmap='inferno', marker='.')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
fig.colorbar(scatter2, label='Depth', shrink=0.7)

ax3 = fig.add_subplot(2, 3, 3)
scatter3 = ax3.scatter(df['x'], df['y'], c=df['Time'], cmap='viridis', marker='.')
fig.colorbar(scatter3, label='Time', shrink=0.7)

ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df['x'], df['y'], df['z'], c=df['Time'], cmap='viridis', marker='.')
fig.colorbar(scatter4, label='Time', shrink=0.7)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df['x'], df['y'], c=df['z'], cmap='inferno', marker='.')
fig.colorbar(scatter5, label='Depth', shrink=0.7)

ax6 = fig.add_subplot(2, 3, 6)
scatter6 = ax6.scatter(df['x'], df['y'], c=df['Time'], cmap='viridis', marker='.')
fig.colorbar(scatter6, label='Time', shrink=0.7)

# andamento chiaro della dipendenza della depth da longitude e latitude, inefficacia del primo plot perchè non c'è una
# dipendenza temporale definita dalla posizione dell'ipocentro

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import plotly.express as px
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns
import numpy.linalg as la

In [ ]:
# AGGIUNTA COLONNE PER I DATI AGGIUNTIVI: Waiting time, Distance, lat, lon, dep
df['Waiting time prev'] = df['Index']
df['Distances prev'] = df['Index']
df['Waiting time'] = df['Index']
df['Distances'] = df['Index']

for i in range(np.shape(df)[0]):
  if df['Prev_event'][i] == -1:
    Time_1 = df['Time'][i]
    Distance_1 = np.array([df['x'][i], df['y'][i], df['z'][i]])
  elif df['Prev_event'][i] > -1:
    Time_1 = df['Time'][df['Prev_event'][i]]
    Distance_1 = np.array([df['x'][df['Prev_event'][i]], df['y'][df['Prev_event'][i]], df['z'][df['Prev_event'][i]]])

  df['Waiting time prev'][i] = df['Time'][i] - Time_1
  df['Distances prev'][i] = math.sqrt(np.sum((np.array([df['x'][i], df['y'][i], df['z'][i]]) - Distance_1)**2))


for i in range(np.shape(df)[0]):
  if df['Index'][i] == 0:
    Time_1 = df['Time'][i]
    Distance_1 = np.array([df['x'][i], df['y'][i], df['z'][i]])
  elif df['Index'][i] > -0:
    Time_1 = df['Time'][i-1]
    Distance_1 = np.array([df['x'][i-1], df['y'][i-1], df['z'][i-1]])

  df['Waiting time'][i] = df['Time'][i] - Time_1
  df['Distances'][i] = math.sqrt(np.sum((np.array([df['x'][i], df['y'][i], df['z'][i]]) - Distance_1)**2))

R = 6371000 # m
df['r'] = df['Index']
for i in range(np.shape(df)[0]):
  df['r'][i] = (df['x'][i]**2 + df['y'][i]**2 + df['z'][i]**2)**0.5

df['lat'] = np.arcsin(df['z'] / df['r'])*180/math.pi
df['lon'] = np.arctan2(df['y'], df['x'])*180/math.pi
df['dep'] = df['r'] - R

df

In [ ]:
fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=df['dep'], cmap='inferno_r', marker='.')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)
fig.colorbar(scatter1, label='depth', shrink=0.5)

ax2 = fig.add_subplot(1, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=df['dep'], cmap='inferno_r', marker='.')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='depth', shrink=0.7)

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
scatter3 = ax3.scatter(df['x'], df['y'], df['z'], c=df['dep'], cmap='inferno_r', marker='.')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.view_init(0,150)
fig.colorbar(scatter3, label='depth', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['lon'], df['lat'], df['dep'], c=df['dep'], cmap='inferno_r', marker='.')
ax1.set_xlabel('longitude')
ax1.set_ylabel('latitude')
ax1.set_zlabel('depth')
ax1.view_init(20,90)
fig.colorbar(scatter1, label='depth', shrink=0.5)

ax2 = fig.add_subplot(1, 3, 2)
scatter2 = ax2.scatter(df['lon'], df['lat'], c=df['dep'], cmap='inferno_r', marker='.')
ax2.set_xlabel('longitude')
ax2.set_ylabel('latitude')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='depth', shrink=0.7)

df_mag4 = df[df['Magnitude'] > 4]
ax3 = fig.add_subplot(1, 3, 3, projection='3d')
scatter3 = ax3.scatter(df_mag4['lon'], df_mag4['lat'], df_mag4['dep'], c=df_mag4['dep'], cmap='inferno_r', marker='.')
ax3.set_xlabel('longitude')
ax3.set_ylabel('latitude')
ax3.set_zlabel('depth')
ax3.set_title('Magnitude > 4')
ax3.view_init(20,90)
fig.colorbar(scatter3, label='depth', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)
fig.colorbar(scatter1, label='Waiting time log', shrink=0.5)

ax2 = fig.add_subplot(1, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Waiting time log', shrink=0.7)

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
scatter3 = ax3.scatter(df['x'], df['y'], df['z'], c=df['Time'], cmap='viridis', marker='.')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.view_init(0,30)
fig.colorbar(scatter3, label='Time', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['lon'], df['lat'], df['dep'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax1.set_xlabel('longtude')
ax1.set_ylabel('latitude')
ax1.set_zlabel('depth')
ax1.view_init(20,90)
fig.colorbar(scatter1, label='Waiting time log', shrink=0.5)

ax2 = fig.add_subplot(1, 3, 2)
scatter2 = ax2.scatter(df['lon'], df['lat'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax2.set_xlabel('longitude')
ax2.set_ylabel('latitude')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Waiting time log', shrink=0.7)

ax3 = fig.add_subplot(1, 3, 3, projection='3d')
scatter3 = ax3.scatter(df['lon'], df['lat'], df['dep'], c=df['Time'], cmap='viridis', marker='.')
ax3.set_xlabel('longitude')
ax3.set_ylabel('latitude')
ax3.set_zlabel('depth')
ax3.view_init(20,90)
fig.colorbar(scatter3, label='Time', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,10))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=df['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)
fig.colorbar(scatter1, label='Magnitude', shrink=0.5)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=np.log(df['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Magnitude log', shrink=0.7)

df_mag3 = df[df['Magnitude'] > 3]
ax3 = fig.add_subplot(2, 3, 3, projection='3d')
scatter3 = ax3.scatter(df_mag3['x'], df_mag3['y'], df_mag3['z'], c=np.log(df_mag3['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.view_init(0,30)
fig.colorbar(scatter3, label='Magnitude log > 3', shrink=0.5)

df_mag4 = df[df['Magnitude'] > 4]
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df_mag4['x'], df_mag4['y'], df_mag4['z'], c=df_mag4['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')
ax4.view_init(0,30)
fig.colorbar(scatter4, label='Magnitude > 4', shrink=0.5)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df_mag4['x'], df_mag4['y'], c=np.log(df_mag4['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax5.set_xlabel('x')
ax5.set_ylabel('y')
ax5.set_aspect(aspect=1)
fig.colorbar(scatter5, label='Magnitude log > 4', shrink=0.7)

df_mag6 = df[df['Magnitude'] > 6]
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
scatter6 = ax6.scatter(df_mag6['x'], df_mag6['y'], df_mag6['z'], c=df_mag6['Magnitude'], cmap='plasma', marker='o', alpha=0.8, vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax6.set_xlabel('x')
ax6.set_ylabel('y')
ax6.set_zlabel('z')
ax6.view_init(0,30)
fig.colorbar(scatter6, label='Magnitude log > 6', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,10))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['lon'], df['lat'], df['dep'], c=df['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax1.set_xlabel('longitude')
ax1.set_ylabel('latitude')
ax1.set_zlabel('depth')
ax1.view_init(20,90)
fig.colorbar(scatter1, label='Magnitude', shrink=0.5)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['lon'], df['lat'], c=np.log(df['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax2.set_xlabel('longitude')
ax2.set_ylabel('latitude')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Magnitude log', shrink=0.7)

df_mag3 = df[df['Magnitude'] > 3]
ax3 = fig.add_subplot(2, 3, 3, projection='3d')
scatter3 = ax3.scatter(df_mag3['lon'], df_mag3['lat'], df_mag3['dep'], c=np.log(df_mag3['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax3.set_xlabel('longitude')
ax3.set_ylabel('latitude')
ax3.set_zlabel('depth')
ax3.view_init(20,90)
fig.colorbar(scatter3, label='Magnitude log > 3', shrink=0.5)

df_mag4 = df[df['Magnitude'] > 4]
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df_mag4['lat'], df_mag4['lon'], df_mag4['dep'], c=df_mag4['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax4.set_xlabel('longitude')
ax4.set_ylabel('latitude')
ax4.set_zlabel('depth')
ax4.view_init(20,90)
fig.colorbar(scatter4, label='Magnitude > 4', shrink=0.5)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df_mag4['lon'], df_mag4['lat'], c=np.log(df_mag4['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax5.set_xlabel('longitude')
ax5.set_ylabel('latitude')
ax5.set_aspect(aspect=1)
fig.colorbar(scatter5, label='Magnitude log > 4', shrink=0.7)

df_mag6 = df[df['Magnitude'] > 6]
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
scatter6 = ax6.scatter(df_mag6['lon'], df_mag6['lat'], df_mag6['dep'], c=df_mag6['Magnitude'], cmap='plasma', marker='o', alpha=0.8, vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax6.set_xlabel('longitude')
ax6.set_ylabel('latitude')
ax6.set_zlabel('depth')
ax6.view_init(20,90)
fig.colorbar(scatter6, label='Magnitude log > 6', shrink=0.5)

plt.tight_layout()

In [ ]:
df_0 = df[df['Prev_event'] > 0]
df_1 = df[df['Prev_event'] == 0]
print(np.shape(df_0))

fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
scatter1_1 = ax1.scatter(df_0['x'], df_0['y'], df_0['z'], c=df_0['Prev_event'], cmap='rainbow', marker='.')
scatter1_2 = ax1.scatter(df_1['x'], df_1['y'], df_1['z'], c='black', marker='.')
fig.colorbar(scatter1_1, label='Prev_event', shrink=0.5)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)

ax2= fig.add_subplot(1, 2, 2, projection='3d')
scatter2_1 = ax2.scatter(df_0['lon'], df_0['lat'], df_0['dep'], c=df_0['Prev_event'], cmap='rainbow', marker='.')
scatter2_2 = ax2.scatter(df_1['lon'], df_1['lat'], df_1['dep'], c='black', marker='.')
fig.colorbar(scatter2_1, label='Prev_event', shrink=0.5)
ax2.set_xlabel('longitude')
ax2.set_ylabel('latitude')
ax2.set_zlabel('depth')
ax2.view_init(20,90)

plt.tight_layout()

In [ ]:
ax1 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.Magnitude, width=600, height=500, opacity=0.5, color_continuous_scale='plasma',
                        center=dict(lat=32, lon=-117), zoom=4, title='Interactive map of the South California earthquakes',
                        mapbox_style='open-street-map')
ax1.show()

ax2 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.dep, width=600, height=500, opacity=0.5, color_continuous_scale='inferno_r',
                        center=dict(lat=32, lon=-117), zoom=4, title='Interactive map of the South California earthquakes',
                        mapbox_style='open-street-map')
ax2.show()

In [ ]:
df['Animation_frame'] = np.round(200*(df['Time']/df['Time'].max()))

In [ ]:
ax1 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.Magnitude, width=600, height=500, opacity=1, color_continuous_scale='plasma', range_color=[df.Magnitude.min(), df.Magnitude.max()],
                        center=dict(lat=33.5, lon=-117), zoom=4, title='Interactive map of the South California earthquakes', animation_frame=df.Animation_frame,
                        mapbox_style='open-street-map')
ax1.show()

ax2 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.dep, width=600, height=500, opacity=1, color_continuous_scale='inferno_r', range_color=[df.dep.min(), df.dep.max()],
                        center=dict(lat=33.5, lon=-117), zoom=4, title='Interactive map of the South California earthquakes', animation_frame=df.Animation_frame,
                        mapbox_style='open-street-map')
ax2.show()

In [ ]:
sns.jointplot(df, x=df.lon, y=df.lat, kind='hist')

In [ ]:
sns.kdeplot(df, x=df.lon, y=df.lat, fill=True, thresh=0, levels=10, cmap='rocket_r')

In [ ]:
sns.jointplot(df, x=df.dep, y=df.Magnitude, kind='hist')

In [ ]:
fig = plt.figure(figsize=(15,5))

ax1 = fig.add_subplot(1,3,1)
ax1.plot(df.Time, df.Index, label='Time vs Index')

ax2 = fig.add_subplot(1,3,2)
ax2.plot(df.Time, df.Magnitude, label='Time vs Magnitude')

ax3 = fig.add_subplot(1,3,3)
ax3.plot(df.Time, df.Magnitude, label='Time vs Magnitude')

# PCA

In [ ]:
PCA_array = np.array(df)[:, 4:7]
print(np.shape(PCA_array))

C = np.cov(PCA_array.T)
sns.heatmap(C)

In [ ]:
U, spectrum, Vt = la.svd(PCA_array[:10000].T)

In [ ]:
l_svd = spectrum**2/(np.shape(df)[0]+1)
V_svd = U

print('Autovalori\t', l_svd/C.trace())
print('Autovettore\t', V_svd[:,0])

In [ ]:
array_newbasis = np.array([la.solve(U, PCA_array[i, :]) for i in range(np.shape(PCA_array)[0])])

In [ ]:
np.shape(array_newbasis)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1, projection='3d')

plt.scatter(array_newbasis[:,0], array_newbasis[:,1], array_newbasis[:,2])

# Resolution (beta)

In [ ]:
#Versione 1
import numpy as np

m = [2, 3, 4, 5]

fig, ax = plt.subplots(1, 4, figsize = (14,7))

for j, i in enumerate(m):
      df2 = df[df['Magnitude'] >= i].sort_values(by = 'Time')
      wt = np.diff(df2['Time'])
      wt[wt<1.e-5] = 1.e-6
      #x = np.logspace(np.log10(np.min(wt)), np.log10(np.max(wt)), 50)
      x = np.linspace(np.min(wt), np.max(wt), 75)
      print(np.min(x), np.max(x))
      print(np.min(wt), np.max(wt))
      
      ax[j].hist(wt, bins = x, edgecolor = 'black')
      #ax[j].set_xscale('log', base = 10)
      ax[j].set_yscale('log', base = 10)
      
plt.show()

In [ ]:
#Versione 2

m = [2, 3, 4, 5]

fig, ax = plt.subplots(1, 4, figsize = (14,7))

for j, i in enumerate(m):
      df2 = df[df['Magnitude'] >= i].sort_values(by = 'Time')
      wt = df2["Time"].diff().to_numpy()
      wt = wt[wt<1.e6]
      x = np.logspace(np.log10(10), np.log10(np.max(wt)), 50)
      #x = np.linspace(np.min(wt), np.max(wt), 75)
      print(np.min(x), np.max(x))
      print(np.min(wt), np.max(wt))
      
      ax[j].hist(wt, bins = x, edgecolor = 'black')
      ax[j].set_xscale('log', base = 10)
      ax[j].set_yscale('log', base = 10)
      
plt.show()

In [ ]:
plt.plot(df['Time'], df['Index'])
plt.show()

In [ ]:
df

In [ ]:
#np.linalg.norm(x2-x1)

fig, ax = plt.subplots(1, 4, figsize = (14,7))

for j, i in enumerate(m):
      df2 = df[df['Magnitude'] >= i][['x', 'y', 'z']]
      coordinates = df2.to_numpy()
      dd = np.linalg.norm(np.diff(coordinates, axis = 0), axis = 1)
      print(dd.shape)
      
      #x = np.logspace(np.log10(np.min(wt)), np.log10(np.max(wt)), 50)
      x = np.linspace(np.min(dd), np.max(dd), 75)
      ax[j].hist(dd, bins = x, edgecolor = 'black')
      #ax[j].set_xscale('log', base = 10)
      #ax[j].set_yscale('log', base = 10)
      
plt.show()

# Point 3. Distributions of waiting times

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import chisquare

First we define the functions we need for data analysis and plots.

The `log_plot` function sets up all parameters we need for plotting histogram in log-log scale:

* To select number of bins, we use the Scott formula: $$N_{\text{bins}}\simeq\frac{3.49\sigma}{n^{\frac{1}{3}}}$$ where $\sigma$ is the standard deviation and $n$ is the number of samples.  
To avoid having too few bins for future regressions, though, we choose a minimum number of bins equal to $10$.

* We generate an equally logarithmically spaced set of points using `np.logspace()` specifying the number of bins we calculated before

* After evaluating bin centers and bin widths we normalize the histogram in order to give a statistical meaning to the distribution.

* We get rid of the empty bins in order for us to be able to evaluate the errors propagating the Poisson error of the unnormalized histogram.

* The function returns the bin centers, the counts per bin and the errors (all in logarithmic scale).

In [ ]:
def log_plot(feature):
    
    #Excluding zeros
    feature = feature[feature != 0]
    
    #Bin edges (in logarithmic scale)
    n_bins = max([np.histogram_bin_edges(np.log10(feature), bins = 'scott').shape[0] - 1, 10])
    
    x = np.logspace(np.log10(feature.min()), np.log10(feature.max()), base = 10, num = n_bins)
    
    #Histogram data
    hist = np.histogram(feature, bins = x)
    bin_centers = (hist[1][1:] + hist[1][:-1]) / 2
    bin_widths = hist[1][1:] - hist[1][:-1]
    
    #Normalization of the histogram
    norm = np.sum(hist[0])
    counts_normalized = hist[0] / (bin_widths * norm)
    
    #Errors
    errors = hist[0]
    
    #Clean of empty bins
    mask = (counts_normalized > 0)
    counts_normalized = counts_normalized[mask]
    bin_centers = bin_centers[mask]
    errors = errors[mask]
    errors = (np.log10(np.e) / (2 * errors)) * np.sqrt(errors)
    
    bins = np.log10(bin_centers)
    counts = np.log10(counts_normalized)
    
    return bins, counts, errors

As discussed later, we use `cut_tails` function to proper select the linearly distributed points (in log-log scale):

* For the left side we first arbitraly drop off all data points a given threshold

* Then, for both sides we recursively compare the pearson coefficient with the one we would obtain getting rid off of the extreme point: if the new value is better than the previous one we reject that point, otherwise we stop the algorithm.

* The function returns bin centers, counts per bin, errors and the pearson coefficients list.

In [ ]:
def cut_tails(b, c, e, threshold):
    
    c = c[b >= threshold]
    e = e[b >= threshold]
    b = b[b >= threshold] 
    
    pearson = [0]
    
    while True:
        new_p = np.corrcoef(b[:-1], y = c[:-1])[0,1]
        if(new_p > pearson[-1] or len(b) <= 9):
            break
        b = b[:-1]
        c = c[:-1]
        e = e[:-1]
        pearson.append(new_p)
        
    while True:
        new_p = np.corrcoef(b[1:], y = c[1:])[0,1]
        if(new_p > pearson[-1] or len(b) <= 9):
            break
        b = b[1:]
        c = c[1:]
        e = e[1:]
        pearson.append(new_p)
        
    return b, c, e, pearson

The `statistic_plots` function allows us to perform a linear regression on the given feature and to evaluate the $\chi^2$.

In [ ]:
def statistic_plots(bins, counts, magnitudes, errors, feature):
    fit_res = []
    chis = []

    fig, ax = plt.subplots(2, len(magnitudes), figsize = (14, 6))

    for i, (x, y, m, e) in enumerate(zip(bins, counts, magnitudes, errors)):

        #Linear regression
        slope, intercept, r, p, se = linregress(x, y)
        fit_res.append((slope, intercept, r, p, se))
        
        #Residuals
        residuals = y - (slope*x+intercept)
        
        #Chi square
        #chis.append(chisquare(y, f_exp = slope*x+intercept, ddof = 2))

        #Plot
        x_axis = np.linspace(x.min(), x.max(), 100)
        ax[0,i].scatter(x, y, label="Data points")
        ax[0,i].plot(x_axis, (slope * x_axis) + intercept, color = 'darkred', label = 'Fit')
        ax[0,i].set_ylabel(f"$\log(P_{m}({feature})$)")
        ax[0,i].set_xlabel(f"$\log({feature})$")
        ax[0,i].grid()

        #Residual plot
        ax[1,i].errorbar(x, residuals, yerr = e, fmt = 'o', capsize = 2.5, label="Residual")
        ax[1,i].hlines(0,x.min(), x.max(), color='darkred', label = 'Fit')
        ax[1,i].grid()

    ax[0,-1].legend()
    ax[1,-1].legend()
    fig.suptitle('Linear regressions and residuals', fontsize = 18)
    plt.tight_layout()
    plt.show()
    #print(chis)
    
    return fit_res, chis

In [ ]:
#Inizializing lists
magnitudes = [2, 3, 4, 5]
bins = []
counts = []
errors = []

for i, m in enumerate(magnitudes):
    
    #Array of waiting times
    wt = np.array(df[df['Magnitude'] >= m][['Time']].sort_values('Time').diff())[1:]
    
    #Prepare data for plot
    b, c, e = log_plot(wt)
    bins.append(b)
    counts.append(c)
    errors.append(e)
    
    #Plot
    plt.scatter(10 ** b, 10 ** c, alpha = 0.75, label = f"m $\geq$ {m}")
    plt.xscale('log', base = 10)
    plt.yscale('log', base = 10)
    
plt.xlabel('Waiting times ($s$)')
plt.ylabel('Probability density ($Hz$)')
plt.title('Distribution of waiting times for different magnitudes')
plt.legend()
plt.grid()
plt.show()

In [ ]:
pearson_list = []

for i, m in enumerate(magnitudes):
    
    bins[i], counts[i], errors[i], pearson = cut_tails(bins[i], counts[i], errors[i], threshold = 2)
    pearson_list.append(pearson)

In [ ]:
fit_res, chis = statistic_plots(bins, counts, magnitudes, errors, 'wt')

In [ ]:
bins   = []
counts = []
errors = []

for i, m in enumerate(magnitudes):
      #Array of siatcnes
      df2 = df[df['Magnitude'] >= m].sort_values('Time')[['x', 'y', 'z']]
      coordinates = df2.to_numpy()
      dd = np.linalg.norm(np.diff(coordinates, axis = 0), axis = 1)
      
      #Prepare data for plot
      b, c, e = log_plot(dd)
      bins.append(b)
      counts.append(c)
      errors.append(e)
      
      #Plot
      plt.scatter(10 ** b, 10 ** c, alpha = 0.75, label = f"m $\geq$ {m}")
      plt.xscale('log', base = 10)
      plt.yscale('log', base = 10)
      
plt.xlabel('Distances ($m$)')
plt.ylabel('Probability density ($m^{-1}$)')
plt.title('Distribution of distances for different magnitudes')
plt.legend()
plt.grid()
plt.show()

In [ ]:
pearson_list = []

for i, m in enumerate(magnitudes):
    
    bins[i], counts[i], errors[i], pearson = cut_tails(bins[i], counts[i], errors[i], threshold = 2)
    pearson_list.append(pearson)

In [ ]:
fit_res, chis = statistic_plots(bins, counts, magnitudes, errors, 'dd')

In [ ]:
magnitudes = [2, 3, 4]
Radii = [10000, 100000, 1000000]

def get_Pmr(df, m, R): #m, R parameters given by the user
    
    dfm = df[df['Magnitude']>= m].sort_values('Time')[['Time', 'x', 'y', 'z']]
    
    #Evaluate the distance between one event and the next one 
    coordinates = dfm.to_numpy()
    dd = np.linalg.norm(np.diff(coordinates, axis = 0), axis = 1)
    dd = np.insert(dd, 0, 0)
    
    dfm['Rel_distance'] = dd
    dfmr = dfm[dfm['Rel_distance'] <= R].sort_values('Time') #select by distance 
    
    wt = np.array(dfmr['Time'].diff())[1:]
    
    return dfmr, dd, wt

bins   = [[] for _ in range(len(Radii))]
counts = [[] for _ in range(len(Radii))]
errors = [[] for _ in range(len(Radii))]

for i, m in enumerate(magnitudes):
    index = 0
    fig, ax = plt.subplots(1, len(Radii), figsize = (14,5))
    
    for j, radius in enumerate(Radii):
        dfmr, dd, wt = get_Pmr(df, m, radius)
        
        b, c, e = log_plot(wt)
        bins[j].append(b)
        counts[j].append(c)
        errors[j].append(e)
        
        ax[index].scatter(10 ** b, 10 ** c, edgecolor = 'black', marker = '.')
        ax[index].set_title(f'Histogram for m $\geqslant$ {m} and R $\leqslant$ {radius}')
        ax[index].set_yscale('log', base = 10)
        ax[index].set_xscale('log', base = 10)
        
        ax[index].set_ylabel(f"$\log(P_{m}(wt)$)")
        ax[index].set_xlabel(f"$\log(wt)$")
        ax[index].grid()
        
        index += 1

    plt.tight_layout()
    
plt.show()

In [ ]:
pearson_list = []

for i, m in enumerate(magnitudes):
    for j, radius in enumerate(Radii):
        bins[j][i], counts[j][i], errors[j][i], pearson = cut_tails(bins[j][i], counts[j][i], errors[j][i], threshold = 2)
        pearson_list.append(pearson)

In [ ]:
for j, radius in enumerate(Radii):

    fit_res, chis = statistic_plots(bins[j], counts[j], magnitudes, errors[j], 'wt')

# Tree structure

In [ ]:
df['Fol event'] = [[] for _ in range(len(df))]

for i in df['Index']:
    if df['Prev_event'][i] > -1:
        df['Fol event'][df['Prev_event'][i]].append(i)
df

In [ ]:
tree = px.treemap(data_frame=df[df['Index'] < 10], parents=df['Prev event'][df['Index'] < 10])
tree.show()